In [2]:
# se não der certo tente pip3
!pip install matplotlib

  Using cached matplotlib-3.9.1.tar.gz (36.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      + meson setup C:\Users\artur\AppData\Local\Temp\pip-install-h9vjd4w3\matplotlib_14b36033c8eb44568f53e1c42a315eb0 C:\Users\artur\AppData\Local\Temp\pip-install-h9vjd4w3\matplotlib_14b36033c8eb44568f53e1c42a315eb0\.mesonpy-2is6pdtx -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\artur\AppData\Local\Temp\pip-install-h9vjd4w3\matplotlib_14b36033c8eb44568f53e1c42a315eb0\.mesonpy-2is6pdtx\meson-python-native-file.ini
      The Meson build system
      Version: 1.5.1
      Source dir: C:\Users\artur\AppData\Local\Temp\pip-install-h9vjd4w3\matplotlib_14b36033c8eb44568f53e1c42a315eb0
      Build dir: C:\Users\artur\AppData\Local\Temp\pip-install-h9vjd4w3\matplotlib_14b36033c8eb44568f53e1c42a315eb0\.mesonpy-2is6pdtx
      Build type: native build
      Program python3 found: YES
      Project name: matplotlib
  

In [ ]:
#bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# SECANTE

# Secante recebe a função f, chutes iniciais x0 e x1, dois erros e iteração máxima
def secante(x0, x1, err1, err2, max_iter):
    # Database
    sec_db = {
        'k': [],
        'Q0': [],
        'Q1': [],  
        'Custo': [],
        'Erro': [] 
        }

    # Contador para as interações
    k = 0
    
    # Se o chute inicial x0 já estiver perto do 0 -> retornar x0
    if abs(f(x0)) < err1:
            insert_db(k, x1, x0, sec_db)
            return x0, sec_db
    
    # Se o chute inicial x1 já estiver perto do 0 -> retornar x1
    if abs(f(x1)) < err1 or abs(x1 - x0) < err2:
            insert_db(k, x0, x1, sec_db)
            return x1, sec_db
    
    # Se não -> executar secante recorrentemente
    while k < max_iter:
        # Executa a iteração
        x2 = x1 - (f(x1) * (x1-x0)) / (f(x1) - f(x0))
        
        # Salvar dados no database
        insert_db(k, x1, x2, sec_db)
        
        # Verifica se convergiu se f(x) é menor que o erro ou |x-x_anterior| é menor que o erro
        if abs(f(x2)) < err1 or abs(x2 - x1) < err2:
            # Se for, retorna x
            return x2, sec_db
        
        # O x atual será o anterior da próxima iteração    
        x0, x1 = x1, x2
        
        # Incrementa o contador
        k = k + 1
    return x2, sec_db

def insert_db(k, x0, x1, db):
    db['k'].append(k)
    db['Q0'].append(round(x0, 5))
    db['Q1'].append(round(x1, 5))
    db['Custo'].append(round(f(x1), 5))
    db['Erro'].append(round(abs(x1 - x0), 5))

# Análise de sinais
def analise_sinais(a, int_max=100):
    # Supondo que o chute inicial esteja próximo da raiz
    b = a + 1
    k = 0
    while (f(a) * f(b)) > 0:
        if k > int_max:
            return print('Error')
        d = (a*f(b) - b*f(b))/(f(b) - f(a))
        if (f(a) * f(d)) > 0: 
            a = d
        else: 
            b = d
    return b
        
# Função que calcula o custo de produção
def f(Q, C=1):
    return C*np.exp(Q) - 4*Q**2 


In [ ]:
# Método de Newton modificado

# Derivada de f(Q)
def df(Q, C):
    return C * np.exp(Q) - 8 * Q

# Método de Newton modificado
def newton_mod(C, Q0, tol=1e-4, max_iter=100):
    # Database do Método de Newton
    newton_db = {
        'k': [],
        'Q0': [],
        'Q1': [],  
        'Custo': [],
        'Erro': [] 
        }
    Q = Q0
    iteracoes = []
    for i in range(max_iter):
        novo_Q = Q - f(Q, C) / df(Q, C)
        insert_db(i, Q, novo_Q, newton_db)
        if abs(novo_Q - Q) < tol:
            return novo_Q, newton_db
        Q = novo_Q
    return Q, newton_db


In [ ]:
# Método do ponto fixo

# Função phi(Q) para o método do ponto fixo
def phi(Q, C):
    return np.sqrt(C * np.exp(Q) / 4)

# Método do ponto fixo
def ponto_fixo(C, Q0, tol=1e-4, max_iter=100):
    mpf_db = {
    'k': [],
    'Q0': [],
    'Q1': [],  
    'Custo': [],
    'Erro': [] 
    }
    Q = Q0
    for i in range(max_iter):
        novo_Q = phi(Q, C)
        insert_db(i, Q, novo_Q, mpf_db)
        if abs(novo_Q - Q) < tol:
            return novo_Q, mpf_db
        Q = novo_Q
    return Q, mpf_db


In [ ]:
# Teste da função
C = 1
Q0 = 0.5
Q1 = analise_sinais(Q0)
err1 = 1e-4
mpf_raiz, mpf_db = ponto_fixo(C, Q0)
newt_raiz, newton_db  = newton_mod(C, Q0)
sec_raiz, sec_db = secante(Q0, Q1, err1, err1, max_iter=100)

df_mpf = pd.DataFrame(mpf_db)
df_newt = pd.DataFrame(newton_db)
df_sec = pd.DataFrame(sec_db)

# Exibir DataFrames
print(f'from Method PF: {mpf_raiz}')
print(df_mpf)
print(f'\nFrom Method Newton: {newt_raiz}')
print(df_newt)
print(f'\nFrom Method Secante: {sec_raiz}')
print(df_sec)

# Função para plotar e salvar o DataFrame como imagem
def salvar_imagem_df(df, nome_arquivo):
    fig, ax = plt.subplots(figsize=(10, 6))  # Tamanho da figura
    ax.axis('tight')
    ax.axis('off')
    tabela = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    tabela.scale(1, 1.5)  # Escala da tabela
    plt.savefig(nome_arquivo, bbox_inches='tight')
    plt.close()

# Salvar imagens dos DataFrames
salvar_imagem_df(df_mpf, 'dados_pf.png')
salvar_imagem_df(df_newt, 'dados_newton.png')
salvar_imagem_df(df_sec, 'dados_secante.png')

print("Imagens salvas com sucesso.")

In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import ttk

# Cria a janela principal
janela1 = tk.Tk()
janela1.title("Quadro resposta")

# Cria o notebook (contêiner de abas)
notebook = ttk.Notebook(janela1)

# Cria frames para cada aba
aba1 = ttk.Frame(notebook)
aba2 = ttk.Frame(notebook)
aba3 = ttk.Frame(notebook)

# Adiciona as abas ao notebook
notebook.add(aba1, text="Ponto Fixo")
notebook.add(aba2, text="Newton Modificado")
notebook.add(aba3, text="Secante Original")

# Empacota o notebook na janela principal
notebook.pack(expand=1, fill="both")

# Adiciona widgets a cada aba
columns_fix = ['k', 'Q0', 'Q1', 'Custo', 'Erro']

label1 = ttk.Treeview(aba1, selectmode='browse', columns=columns_fix, show='headings')
label1.pack(pady=10, padx=10)

label2 = ttk.Treeview(aba2, selectmode='browse', columns=columns_fix, show='headings')
label2.pack(pady=10, padx=10)

label3 = ttk.Treeview(aba3, selectmode='browse', columns=columns_fix, show='headings')
label3.pack(pady=10, padx=10)

# Insere dados na Treeview
def construct(label, db):
    label.column('k', width=20, minwidth=10, stretch=NO)
    label.heading('#1', text='k')

    label.column('Q0', width=100, minwidth=50, stretch=NO)
    label.heading('#2', text='Q0')

    label.column('Q1', width=100, minwidth=50, stretch=NO)
    label.heading('#3', text='Q1')

    label.column('Custo', width=100, minwidth=50, stretch=NO)
    label.heading('#4', text='Custo')

    label.column('Erro', width=100, minwidth=50, stretch=NO)
    label.heading('#5', text='Erro')

    #label.grid(row=0, column=0)

    elementos = []
    for i in range(0, len(db['k'])):
        for key, value in db.items():
            elementos.append(db[key][i])
        label.insert("", END, values=elementos, tag=i)
        elementos = []

construct(label1, sec_db)
construct(label2, newton_db)
construct(label3, mpf_db)


#janela1.mainloop()